# 세미나 자료

기본적으로 offset_mapping과 overflow_to_sample_mapping에 대한 개념을 알고 있다고 가정하고 세미나를 진행합니다!

In [13]:
import os
import torch
import numpy as np
from numpy import dot
from numpy.linalg import norm
from functools import partial
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_from_disk, DatasetDict, Dataset
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
from konlpy.tag import Mecab
import random
import re

### 사용될 constants

In [14]:
max_seq_length=384
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Question과 Context에 대해서 Embedding 값을 구해줄 Encoders

In [15]:
p_encoder = AutoModel.from_pretrained('kiyoung2/roberta-large-qaconv-sds', use_auth_token=True).to(device)
q_encoder = AutoModel.from_pretrained('kiyoung2/roberta-large-qaconv-sds', use_auth_token=True).to(device)

Some weights of the model checkpoint at kiyoung2/roberta-large-qaconv-sds were not used when initializing RobertaModel: ['qa_outputs.convs.0.layer_norm.weight', 'qa_outputs.convs.0.layer_norm.bias', 'qa_outputs.convs.2.conv2.bias', 'qa_outputs.convs.1.conv2.weight', 'qa_outputs.convs.1.conv1.weight', 'qa_outputs.convs.3.conv2.bias', 'qa_outputs.convs.4.conv1.bias', 'qa_outputs.convs.3.conv1.weight', 'qa_outputs.convs.2.conv1.bias', 'qa_outputs.convs.3.conv2.weight', 'qa_outputs.convs.2.layer_norm.weight', 'qa_outputs.convs.2.conv1.weight', 'qa_outputs.convs.3.conv1.bias', 'qa_outputs.convs.0.conv1.bias', 'qa_outputs.convs.3.layer_norm.bias', 'qa_outputs.qa_output.bias', 'qa_outputs.convs.4.conv2.weight', 'qa_outputs.convs.2.conv2.weight', 'qa_outputs.convs.4.conv1.weight', 'qa_outputs.convs.1.conv1.bias', 'qa_outputs.convs.0.conv1.weight', 'qa_outputs.convs.1.layer_norm.weight', 'qa_outputs.convs.0.conv2.weight', 'qa_outputs.convs.4.layer_norm.bias', 'qa_outputs.convs.3.layer_norm.weig

### Masked Dataset을 만들어줘!!!

In [35]:
def make_emb_mask_dataset(dataset_path):
    tokenizer = AutoTokenizer.from_pretrained('kiyoung2/roberta-large-qaconv-sds', use_auth_token=True)
    
    # 데이터 불러오기!!
    raw_train_dataset = load_from_disk(os.path.join(dataset_path, "train_dataset"))['train']
    raw_val_dataset = load_from_disk(os.path.join(dataset_path, "train_dataset"))['validation']
    
    # 비효율적이야... 하지만 offset_mapping과 sample_mapping이 필요해..
    tokenized_c = tokenizer(raw_train_dataset['context'], return_tensors='pt', truncation=True, max_length=max_seq_length, stride=128,return_overflowing_tokens=True,return_offsets_mapping=True,padding="max_length")
    
    offset_mapping = tokenized_c.pop("offset_mapping")
    sample_mapping = tokenized_c.pop("overflow_to_sample_mapping")
    
    # 영차영차! 데이터를 만들자!! 
    ext_prepare_train_features_flatten_trunc = partial(_ext_prepare_train_features_flatten_trunc, tokenizer = tokenizer)
    new_q = raw_train_dataset.map(
        ext_prepare_train_features_flatten_trunc,
        batched=True,
        remove_columns=raw_train_dataset.column_names
    )
    
    # batch 단위의 처리를 위해서 Dataloader를 만들어보자!!
    Tensor_dataset = TensorDataset(
        tokenized_c['input_ids'], tokenized_c['attention_mask'], tokenized_c['token_type_ids'],
        torch.tensor(new_q['ids']), torch.tensor(new_q['attention']), torch.tensor(new_q['token_type']),
        torch.tensor(new_q['answer'])
    )

    train_dataloader = DataLoader(
        Tensor_dataset,
        batch_size=2
    )
    
    # 데이터를 만들었으니 Masking을 해보자!!
    masked_dataset = mask_word_with_emb(train_dataloader, tokenizer, offset_mapping, sample_mapping, raw_train_dataset)
    
    new_dataset = DatasetDict({
        'train':masked_dataset,
        'validation':raw_val_dataset
    })
    
    return new_dataset

### Truncation! 때문에 Question과 context, answer에 mapping 하는게 힘들어!!

In [36]:
def _ext_prepare_train_features_flatten_trunc(examples, tokenizer):
    new_tokenized_ids=[]
    new_att =[]
    new_token_type=[]
    new_answer = []
    
    texts = [text['text'][0] for text in examples['answers']]
    
    tokenized_q = tokenizer(examples['question'], return_tensors='pt', truncation=True, max_length=max_seq_length, padding="max_length")
    tokenized_c = tokenizer(examples['context'], return_tensors='pt', truncation=True, max_length=max_seq_length, stride=128,return_overflowing_tokens=True,return_offsets_mapping=True,padding="max_length")
    tokenized_a = tokenizer(texts, return_tensors='pt', max_length=100, padding="max_length")
    
    sample_mapping = tokenized_c.pop("overflow_to_sample_mapping")
    
    # truncation된 데이터에 맞춰서 answer와 question를 추가해주자!!
    for i in tqdm(sample_mapping):
        new_tokenized_ids.append(tokenized_q['input_ids'][i].tolist())
        new_att.append(tokenized_q['attention_mask'][i].tolist())
        new_token_type.append(tokenized_q['token_type_ids'][i].tolist())
        new_answer.append(tokenized_a['input_ids'][i].tolist())
    
    return { 'ids':new_tokenized_ids,
            'attention':new_att,
            'token_type':new_token_type,
            'answer':new_answer }

### 본격적으로 Masking 해볼까?

In [37]:
def mask_word_with_emb(dataloader, tokenizer, offset_mapping, sample_mapping, train_dataset):
    new_ids=[]
    mask_token = tokenizer.mask_token_id
    
    # similarity를 계산할 때, 계산되면 안되는 special token들
    ignore_tokens = [tokenizer.pad_token_id, 
                     tokenizer.unk_token_id,
                     tokenizer.cls_token_id, 
                     tokenizer.sep_token_id]
    
    with tqdm(dataloader, unit="batch") as tepoch:
        for batch in tepoch:
            labels = batch[0].clone()
            
            # tokenized context 정보
            p_inputs={
                "input_ids": batch[0].to(device),
                "attention_mask": batch[1].to(device),
                "token_type_ids": batch[2].to(device)
            }
            
            # tokenized question 정보
            q_inputs = {
                "input_ids": batch[3].to(device),
                "attention_mask": batch[4].to(device),
                "token_type_ids": batch[5].to(device)
            }
            
            # masking하려는 단어가 answer인지 아닌지 판단하려면 필요하겠지?
            answers = [tokenizer.decode(i, skip_special_tokens=True) for i in batch[6]]
            
            # 위에 선언했던 무시해야 할 special token의 위치를 표시할 matrix
            matrix = torch.full(labels.shape, True)
            
            # list에 포함된 token이랑 같으면 matrix의 값을 False로 바꾸자!
            for ignore_token in ignore_tokens:
                ignore_mask = labels.eq(ignore_token)
                matrix.masked_fill_(ignore_mask, value=False)
            
            # 0 -> 각 token들에 대한 embedding 값
            # 1 -> 전체에 대한 embedding 값
            p_outputs = p_encoder(**p_inputs)[0]
            q_outputs = q_encoder(**q_inputs)[1]
            
            batch_size = p_outputs.shape[0]
            
            # (batch_size, 1024) -> (batch_size, 1, 1024)
            q_outputs = q_outputs.view(batch_size,1,-1)
            # (batch_size, 384, 1024) -> (batch_size, 1024, 384)
            p_outputs = torch.transpose(p_outputs.view(batch_size, max_seq_length, -1), 1, 2)
            
            #batch 단위 matmul
            #(batch_size, 1, 384) -> (batch_size, 384)
            sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()
            # 모양이 제대로 잡혔지만, 확인 차 !
            # (batch_size, 384) -> (batch_size, 384)
            sim_scores = sim_scores.view(batch_size, -1)
            # special token 값이 softmax에 반영되지 않게!
            sim_scores[~matrix] = -100
            # 자~ 구해보자~ similarity!!
            sim_scores = F.log_softmax(sim_scores, dim=1)
            
            # similarity가 큰 순서대로 sorting을 해주고, 점수가 큰거부터 masking!
            for idx, score in enumerate(sim_scores):
                sorted_score, sorted_idx = torch.sort(score, descending=True)
                for i in range(2):
                    p_inputs['input_ids'][idx] = make_mask_word(tokenizer, p_inputs['input_ids'][idx], answers[idx], sorted_idx[i])
                new_ids.append(p_inputs['input_ids'][idx].tolist())
    
    #어!? masking 해주니까 answer 위치가 바뀌네!? 잡아주자!
    origin_answers = train_dataset['answers']
    origin_start = [data['answer_start'][0] for data in train_dataset['answers']]
    origin_text = train_dataset['context']
    
    # token 단위에서 처리한 masking을 원래 dataset에 반영해주자!
    for list_idx, offset_list in enumerate(offset_mapping):
        for idx, offsets in enumerate(offset_list):
            # 같은 거는 special token
            if offsets[0] == offsets[1]:
                continue
            else:
                #아닐 경우 mask!
                if new_ids[list_idx][idx] == tokenizer.mask_token_id:
                    for i in range(offsets[0], offsets[1]):
                        origin_list_idx = sample_mapping[list_idx]
                        origin_text[origin_list_idx] = list(origin_text[origin_list_idx])
                        origin_text[origin_list_idx][i] = '∬'
                        origin_text[origin_list_idx] = "".join(origin_text[origin_list_idx])

    # mask 위치를 다시 잡아주자!
    for i, text in enumerate(origin_text):
        origin_text[i] = text[:origin_start[i]] + '[ans]' + text[origin_start[i]:]
    
    # 편- 안-
    for idx, origin in enumerate(origin_text):
        origin_text[idx] = re.sub('∬+',tokenizer.mask_token, origin)
        origin_answers[idx]['answer_start'] = [origin_text[idx].find('[ans]')]
        origin_text[idx] = re.sub('\[ans\]','', origin_text[idx])
    
    # 휴 끝났어유~
    return Dataset.from_dict({
        'title': train_dataset['title'],
        'context': origin_text,
        'question': train_dataset['question'],
        'id': train_dataset['id'],
        'answers': origin_answers,
        'document_id': train_dataset['document_id'],
        '__index_level_0__' : train_dataset['__index_level_0__'],
    })

### token이 포함된 단어를 찾아보자!!

In [38]:
def make_mask_word(tokenizer, ids, answer, idx):
    # 와 ~ tensor 값은 -, + 안되는지 모르고 한참을 디버깅했다..
    front_idx = int(idx)
    back_idx = int(idx)
    
    tokens = tokenizer.convert_ids_to_tokens(ids)
    
    # 단어 시작점을 찾아주자!
    while True:
        if tokens[front_idx][:2] == "##":
            front_idx -= 1
        elif (len(tokens[front_idx])<=2 or tokens[front_idx][:2]!="##"):
            break
        else: 
            front_idx -= 1
    # 단어 끝점을 찾아주자!
    while True:
        if (len(tokens[back_idx+1])<=2) or (tokens[back_idx+1][:2]!="##"):
            break
        else:
            back_idx+=1
    
    # ##는 시져시져~
    word =  re.sub('##','',''.join(tokens[front_idx:back_idx+1]))
    
    # 정답이 아니면 masking!!!!! 받아랏!
    if answer not in word:
        for idx in range(front_idx,back_idx+1):
            tokens[idx] = tokenizer.mask_token
    
    result = torch.tensor(tokenizer.convert_tokens_to_ids(tokens))

    return result

## 자 돌려보자!!

In [39]:
result_dataset = make_emb_mask_dataset("./data/aistage-mrc/")

Loading cached processed dataset at data/aistage-mrc/train_dataset/train/cache-d171d7709cac2c45.arrow


  0%|          | 0/3647 [00:00<?, ?batch/s]

## 너도 이제 마스킹 고수!!

In [41]:
result_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [42]:
result_dataset['train']['context'][:5]

['미국 상의원 또는 미국 상원(United States Senate)은 [MASK] 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다. 미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다[MASK] 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 [MASK] 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 [MASK] 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 "'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 1950년대이다. 2차 세계대전을 마치고, 6.25전쟁의 시기로 유럽은 전후 재건에 집중하고, 유럽 제국주의의 식민지가 독립하여 아프리카, 아시아